In [1]:
import numpy as np
import pandas as pd

import bokeh_catplot

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()


Loading BokehJS ...

In [2]:
df = pd.read_csv('C:/Users/yang1/git/bootcamp/data/ctx_lineplot_1july2020.csv',na_values='*')
with open('C:/Users/yang1/git/bootcamp/data/ctx_lineplot_1july2020.csv') as f:
    for _ in range(10): #first 10 lines
        print(f.readline())
        

        
        

Subject,ID,ctx (pg/ml)

1,0,222

1,3,336

1,6,357

1,9,347

1,12,407

1,15,

1,18,318

1,21,356

1,24,746



In [3]:
def scatter(
    data=None,
    cat=None,
    x=None,
    y=None,
    p=None,
    palette=None,
    show_legend=True,
    click_policy="hide",
    marker_kwargs={},
    **kwargs,
):
    #name axes automatically?
    if "ID" not in kwargs:
        kwargs["ID"] = x
    if "ctx (pg/ml)" not in kwargs:
        kwargs["ctx (pg/ml)"] = y

    # Default palette
    if palette is None:
        palette = colorcet.b_glasbey_category10
    elif type(palette) == str:
        palette = [palette]

    # Instantiate figure
    if p is None:
        p = bokeh.plotting.figure(**kwargs)

    # Build plot (not using color factors) to enable click policies
    for i, (name, g) in enumerate(data.groupby(cat, sort=False)):
        marker_kwargs["color"] = palette[i % len(palette)]
        marker_kwargs["legend_label"] = str(name)
        p.circle(source=g, x=x, y=y, **marker_kwargs)

    if show_legend:
        p.legend.click_policy = click_policy
    else:
        p.legend.visible = False

    return p


In [4]:
#drop subjects with NaN for timepoint mo12 and baseline
s1 = df.loc[df['ID']==12, ['Subject', 'ctx (pg/ml)']]
s0 = df.loc[df['ID']==0, ['Subject', 'ctx (pg/ml)']]
baseline=s0.dropna()
month12=s1.dropna()


In [5]:
#rename columns
month12_new = month12.rename(columns={'ctx (pg/ml)': 'ctx mo12'}, index={'ONE': 'one'})
baseline_new = baseline.rename(columns={'ctx (pg/ml)': 'ctx baseline'}, index={'ONE': 'one'})



In [6]:
#inner join two datasets and remname second column as month 12 
inner_join_df= pd.merge(baseline_new, month12_new, on='Subject', how='inner')

#separate them into respective frames
baseline_only = inner_join_df['ctx baseline']
baseline_only
month12_only = inner_join_df['ctx mo12']
month12_only

0     407.0
1      95.0
2     527.0
3     425.0
4     131.0
5     160.0
6     271.0
7     237.0
8     794.0
9     328.0
10    202.0
11    271.0
12    250.0
13    263.0
14    302.0
15    282.0
16    200.0
17    212.0
18    372.0
19    306.0
20     49.0
21    341.0
22    471.0
23    280.0
24    148.0
25    291.0
26    364.0
27    189.0
28    351.0
29    519.0
30    456.0
31    433.0
32    133.0
33    207.0
34    432.0
35    249.0
Name: ctx mo12, dtype: float64

In [7]:
#draw bootstraps
def draw_bs_pairs(x, y, rg):
    """Draw pairs of data points out of `x` and `y`"""
    # Get indices of bootstrap sample
    bs_inds = rg.choice(np.arange(len(x)), len(x), replace=True)

    return x[bs_inds], y[bs_inds]

In [12]:
#determine %change mean? between baseline and month 12
n_reps = 2000
rg = np.random.default_rng()
ratio_bs_reps = np.empty(n_reps)

for i in range(n_reps):
    bl, m12 = draw_bs_pairs(baseline_only, month12_only, rg)
    ratio_bs_reps[i] = np.mean(((m12-bl)/bl))
    


In [11]:
np.percentile(ratio_bs_reps, [2.5, 97.5])

array([0.30284067, 0.70469249])